In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append("../..")
sys.path.append("../../..")

import pandas as pd
import numpy as np
from mlex import DataReader, get_pcpe_dtype_dict, pcpe_preprocessing_read_func

In [ ]:
import torch

target_column = 'I-d'
filter_data = {'NATUREZA_LANCAMENTO': 'C'}
path = r'/data/pcpe/pcpe_03.csv'

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

reader = DataReader(path, target_columns=[target_column], filter_dict=filter_data, dtype_dict=get_pcpe_dtype_dict(), preprocessing_func=pcpe_preprocessing_read_func)
reader.fit(insert_noise=True)
X = reader.fit_transform(X=None)
y = reader.get_target()
print(y)

In [ ]:
import sys
import os

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from copy import deepcopy
from mlex.features.sequences import SequenceTransformer
from mlex import FeatureStratifiedSplit
from mlex import PreProcessingTransformer
from mlex import DataReader
from mlex import StandardEvaluator
from mlex import F1MaxThresholdStrategy
from mlex.models.models import RNNModel


threshold_strategy = 'f1max'
threshold_selection = F1MaxThresholdStrategy()
sequence_lengths = [10, 20, 30, 40, 50]
batch_size = 32
column_to_stratify = 'CPF_CNPJ_TITULAR'
hidden_size = 10
num_layers = 1
num_classes = 1
epochs = 30
patience = 5
target_column = 'I-d'
filter_data = {'NATUREZA_LANCAMENTO': 'C'}
sequences_compositions = ['temporal', 'account', 'individual']
sequence_column_dict = {'temporal': None, 'account': 'CONTA_TITULAR', 'individual': 'CPF_CNPJ_TITULAR'}
iterations = 10
path = r'/data/pcpe/pcpe_03.csv'

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

reader = DataReader(path, target_columns=[target_column], filter_dict=filter_data, dtype_dict=get_pcpe_dtype_dict(), preprocessing_func=pcpe_preprocessing_read_func)
X, y = reader.get_X_y()

final_models = []

for sequence_composition in sequences_compositions:
    print(f"experiment sequence {sequence_composition}")
    sequence_column = sequence_column_dict[sequence_composition]
    for sequence_length in sequence_lengths:
        print(f"sequence length {sequence_length}")
        for i in range(iterations):
            print(f"iteration {i+1}")
            # Initialize splitter
            splitter_tt = FeatureStratifiedSplit(stratify_column=column_to_stratify, split_proportion=0.3)
            splitter_tt.fit(X, y)
            # Get splits
            X_train, y_train, X_test, y_test = splitter_tt.transform(X, y)

            splitter_tv = FeatureStratifiedSplit(stratify_column=column_to_stratify, split_proportion=0.1)
            splitter_tv.fit(X_train, y_train)
            # Get splits
            X_train, y_train, X_val, y_val = splitter_tv.transform(X_train, y_train)

            group_train, group_val, group_test = (None, None, None)
            if sequence_composition != 'temporal':
                group_train = X_train.loc[:,sequence_column].values
                group_val = X_val.loc[:,sequence_column].values
                group_test = X_test.loc[:, sequence_column].values

            preprocessor = PreProcessingTransformer(target_column=[target_column])

            X_train_array = preprocessor.transform(X_train, y_train, insert_noise=True)
            y_train_array = preprocessor.get_target()
            features_names_train = preprocessor.get_feature_names_out()